In [ ]:
import tensorflow as tf
import os
print tf.__version__

## Set gcloud commands' variables

In [ ]:
PROJECT = 'ksalama-gcp-playground'
BUCKET = 'ksalama-gcs-cloudml'
REGION = 'europe-west1'

os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

## Deploy the model

In [ ]:
%bash
MODEL_NAME="taxifare_estimator"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/ml-models/taxifare/dnn-combined-regression-small/export/Servo | tail -1)
echo $MODEL_LOCATION

# echo "Deleting version $MODEL_VERSION for model $MODEL_NAME"
# gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}

# echo "Deleting model $MODEL_NAME"
# gcloud ml-engine models delete ${MODEL_NAME}

# echo "Creating model $MODEL_NAME"
# gcloud ml-engine models create ${MODEL_NAME} --regions $REGION

echo "Creating version $MODEL_VERSION for model $MODEL_NAME"
gcloud ml-engine versions create ${MODEL_VERSION} --model=${MODEL_NAME} --origin ${MODEL_LOCATION}

## Predictions for new data points

In [ ]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

request_data = {'instances':
  [
      {
        'dayofweek': 'Fri',
        'hourofday': 2,
        'pickuplon': -73.885262,
        'pickuplat': 40.773008,
        'dropofflon': -73.987232,
        'dropofflat': 40.732403,
        'passengers': 6,

      },
      {
        'dayofweek': 'Sat',
        'hourofday': 2,
        'pickuplon': -73.885262,
        'pickuplat': 42.773008,
        'dropofflon': -73.987232,
        'dropofflat': 40.732403,
        'passengers': 6,

      }
    
  ]
}

model_url = 'projects/{}/models/{}/versions/{}'.format(PROJECT, 'taxifare_estimator', 'v2')
response = api.projects().predict(body=request_data, name=model_url).execute()
print "response={0}".format(response)